In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

data = pd.read_csv("../../data/WorkoutExport.csv")

# Read and Reshape Data

I'm not importing this data from a FitBod export, which tracks each set as a row, uses multipliers, doesn't contain calculated volumes, and does requires some unit conversions.

In [2]:
# data["Date"] = pd.to_datetime(data["Date"])
data.dtypes

Date            object
Exercise        object
Reps             int64
Weight(kg)     float64
Duration(s)    float64
Distance(m)    float64
Incline        float64
Resistance     float64
isWarmup          bool
Note           float64
multiplier     float64
dtype: object

In [3]:
data.info()
"""
    These don't provide much value as I'm calculating total volume (including warmups),
    and resistance/notes are not features I utilize in the FitBod application
"""
data.drop(["Note", "isWarmup", "Resistance"], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1251 non-null   object 
 1   Exercise     1251 non-null   object 
 2   Reps         1251 non-null   int64  
 3   Weight(kg)   1251 non-null   float64
 4   Duration(s)  1251 non-null   float64
 5   Distance(m)  1251 non-null   float64
 6   Incline      1251 non-null   float64
 7   Resistance   1251 non-null   float64
 8   isWarmup     1251 non-null   bool   
 9   Note         0 non-null      float64
 10  multiplier   1251 non-null   float64
dtypes: bool(1), float64(7), int64(1), object(2)
memory usage: 99.1+ KB


In [4]:
data.head()

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,multiplier
0,2022-02-25 23:14:47 +0000,Walking,0,0.000000,1170.638832,1891.400415,0.0,0.0
1,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,12,22.679645,0.000000,0.000000,0.0,2.0
2,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,12,22.679645,0.000000,0.000000,0.0,2.0
3,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,10,22.679645,0.000000,0.000000,0.0,2.0
4,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,8,22.679645,0.000000,0.000000,0.0,2.0


Now that we have a clean dataframe, let's convert weight to pounds, calculate a volume for each set (row), and set a "Sets" column to 1 across the board for summing later.

In [5]:
kg_to_lb_conversion = 2.20462
data["Weight(lb)"] = data["Weight(kg)"] * kg_to_lb_conversion
data["Volume(lb)"] = data["Weight(lb)"] * data["multiplier"] * data["Reps"]
data["Sets"] = 1

this_week = data.loc[(data["Date"] >= "2022-02-20")]
# this_week.drop(["Weight(kg)"], axis=1, inplace=True)

this_week.head()

,Date,Exercise,Reps,Weight(kg),Duration(s),Distance(m),Incline,multiplier,Weight(lb),Volume(lb),Sets
0,2022-02-25 23:14:47 +0000,Walking,0,0.000000,1170.638832,1891.400415,0.0,0.0,0.0,0.0,1
1,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,12,22.679645,0.000000,0.000000,0.0,2.0,50.0,1200.0,1
2,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,12,22.679645,0.000000,0.000000,0.0,2.0,50.0,1200.0,1
3,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,10,22.679645,0.000000,0.000000,0.0,2.0,50.0,1000.0,1
4,2022-02-25 22:24:10 +0000,Dumbbell Bench Press,8,22.679645,0.000000,0.000000,0.0,2.0,50.0,800.0,1


# Weekly Workout Stats

#### My split:
- Back, Biceps, Shoulders (Day 1)
- Chest, Triceps (Day 2)
- Quads, Hamstrings, Calves, Glutes (Day 3)

#### My goals:
- Min: 10, Max: 20 sets per muscle gorup, per training week (2 splits)
- Min: 8, Max: 12 reps per set
- Qualitative (and untracked, currently): Achieve maximum hyperbolic stimulus through improved mind-muscle connection, proper form, and progressive overloading within the safezone of 3-0 RIR (reps in reserve)

#### Methods:

To achieve peak hypertrophic stimulus, each set will be taken between 3-0 RIR, or close to, if not at, failure. Within these parameters, I will aim for an average rep range of 8-12 RPS (reps per set)

In [6]:
tw_pivot = this_week.groupby("Exercise").sum()

# Will worry about cardio tracking later
tw_pivot.drop(columns=["Duration(s)", "Distance(m)", "Incline", "multiplier", "Weight(kg)", "Weight(lb)"], inplace=True)
tw_pivot

,Reps,Volume(lb),Sets
Exercise,,,
Back Squat,70,8650.0,7
Barbell Hip Thrust,20,1400.0,2
Barbell Shrug,48,4080.0,4
Bicycle Crunch,10,0.0,1
Cable Tricep Pushdown,93,3432.5,10
Dumbbell Bench Press,107,10140.0,10
Dumbbell Bicep Curl,72,4220.0,8
Dumbbell Fly,101,4460.0,10
Dumbbell Front Raise,74,2680.0,8


In [7]:
# Takes array of strings
def get_data(exercises):
    return tw_pivot.loc[exercises]

### Back, Shoulder, Biceps (Day 1)

In [8]:
day1 = get_data([
    "Dumbbell Row",
    "V-Bar Pulldown",
    "Dumbbell Front Raise",
    "Dumbbell Shoulder Press",
    "Dumbbell Bicep Curl",
    "Hammer Curls",
    "Barbell Shrug",
    "Dumbbell Upright Row"
])
day1

,Reps,Volume(lb),Sets
Exercise,,,
Dumbbell Row,99,9190.0,10
V-Bar Pulldown,98,10290.0,10
Dumbbell Front Raise,74,2680.0,8
Dumbbell Shoulder Press,76,4560.0,8
Dumbbell Bicep Curl,72,4220.0,8
Hammer Curls,60,3250.0,8
Barbell Shrug,48,4080.0,4
Dumbbell Upright Row,42,2100.0,4


In [9]:
# Total Volume
day1["Volume(lb)"].sum()

40370.0

### Chest, Triceps (Day 2)

In [10]:
day2 = get_data([
    "Dumbbell Bench Press",
    "Dumbbell Fly",
    "Cable Tricep Pushdown",
    "Dumbbell Kickbacks"
])
day2

,Reps,Volume(lb),Sets
Exercise,,,
Dumbbell Bench Press,107,10140.0,10
Dumbbell Fly,101,4460.0,10
Cable Tricep Pushdown,93,3432.5,10
Dumbbell Kickbacks,116,4040.0,10


In [11]:
# Total Volume
day2["Volume(lb)"].sum()

22072.5

### Quads, Hamstrings, Calves, Glutes (Day 3)

In [12]:
day3 = get_data([
    "Back Squat",
    "Romanian Deadlift",
    "Barbell Hip Thrust"
])
day3

,Reps,Volume(lb),Sets
Exercise,,,
Back Squat,70,8650.0,7
Romanian Deadlift,68,7820.0,7
Barbell Hip Thrust,20,1400.0,2


In [13]:
# Total Volume
day3["Volume(lb)"].sum()

17870.0

# Weekly Training Reflections

TODO: 2022-02-27